# 상품 정보 크롤링

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# 웹드라이버 실행
# url = 'https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sub_category_id=474&sort=200'  # 상의
# url = 'https://zigzag.kr/categories/-1?title=%EC%95%84%EC%9A%B0%ED%84%B0&category_id=-1&middle_category_id=436&sort=200'  # 아우터
url = 'https://zigzag.kr/categories/-1?title=%ED%8C%AC%EC%B8%A0&category_id=-1&middle_category_id=547&sort=200' #팬츠
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# 스크롤하여 데이터 수집
scrolls = 9  # 원하는 만큼 스크롤 (조정 가능)
collected_links = set()  # 중복 방지를 위한 set()

for _ in range(scrolls):
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # 페이지가 로딩될 시간을 줌
    
    # 페이지 HTML 업데이트
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 상품 링크 수집 (새로운 상품이 로딩되었을 가능성이 있으므로 계속 추가)
    links = soup.select("#__next > div.zds-themes.light-theme > div.css-xrh5h3.edi339y0 > main > section.css-j2fe9p.eeob9mf0 > div a")
    
    for link in links:
        product_url = link.get('href')
        if product_url and product_url.startswith('/catalog/products/'):
            collected_links.add(f'https://zigzag.kr{product_url}')  # 중복 제거 위해 set에 추가

# 최종 수집된 링크 확인
print(f"총 {len(collected_links)}개의 상품 링크를 수집했습니다.")

# 수집된 정보를 저장할 리스트
data = []

# 상품 상세 정보 크롤링
for idx, full_url in enumerate(list(collected_links)):  # 모든 상품 링크를 크롤링
    driver.get(full_url)
    time.sleep(2)
    
    item_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        brand_name = item_page.select_one('.css-gwr30y').text.strip()
        item_name = item_page.select_one('h1.css-1n8byw.e14n6e5u1').text.strip()
        origin_price = item_page.select_one('.css-14j45be.e1yx2lle2').text.strip()
        discount_price = item_page.select_one('.css-no59fe.e1ovj4ty1').text.strip()
        review_count = item_page.select_one('span.css-1ovjo5n').text.strip()
        item_score_avg = item_page.select_one('.css-1hld56p.e71452m2').text.strip()
        brand_keyword = [span.text for span in item_page.select("#__next > div.zds-themes.light-theme > div.css-xrh5h3.edi339y0 > div > div.shop_row > div > div > div > div > div:nth-child(2) > div.css-2jjng0.eq4fxvi7 span")]
        
        print(brand_name, item_name, origin_price, discount_price, review_count, item_score_avg, brand_keyword)
        
        # 수집한 데이터를 리스트에 추가
        data.append({
            '브랜드': brand_name,
            '상품명': item_name,
            '원가': origin_price,
            '할인 가격': discount_price,
            '리뷰 개수': review_count,
            '평점': item_score_avg,
            '키워드': ', '.join(brand_keyword)
        })
        
        print(len(data))

    except AttributeError:
        print(f"[{idx+1}] 데이터가 부족한 상품입니다. URL: {full_url}")
    
    time.sleep(2)

# 크롤링된 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame을 CSV 파일로 저장
df.to_csv(r'D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\pants_info.csv', index=False, encoding='utf-8')

# 브라우저 종료
driver.quit()

print(f"총 {len(df)}개의 상품 정보를 CSV로 저장했습니다.")

## 리뷰 크롤링

In [ ]:
import time
import pandas as pd
import csv
import re
import os 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 웹드라이버 실행
url = 'https://zigzag.kr/search'
driver = webdriver.Chrome()

# 데이터 불러오기
df = pd.read_csv(r'D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\top_info.csv', encoding='utf-8')
product_names = df['상품명'].tolist()

# 파일이 없을 때만 새로 생성
file_path = r"D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\top_reviews.csv"

# 항상 새로 파일을 덮어쓰도록 수정
with open(file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["브랜드명", "상품명", "리뷰어", "날짜", "사이즈", "퀄리티", "색감", "키", "몸무게", "상의", "리뷰"])  
    print("📂 CSV 파일 덮어쓰기 완료!")
    
    for product in product_names:
        driver.get(url)
        time.sleep(1)
    
        try:
            # 검색창 찾기
            search_box = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/main/div[1]/nav/div/input')
            search_box.clear()
            search_box.send_keys(product)
            search_box.send_keys(Keys.ENTER)
            time.sleep(2)
            
            # 첫 번째 상품의 링크 가져오기
            link = driver.find_element(By.CSS_SELECTOR, '#__next > div > div.css-xrh5h3.edi339y0 > main > div.css-1ad8qtm.basic > div > div > div > div > div > div > div > a')
            product_link = link.get_attribute("href")
            driver.get(product_link)
            time.sleep(1)
            
            # 브랜드명, 상품명 가져오기
            brand_name = driver.find_element(By.CSS_SELECTOR, '.css-gwr30y').text.strip()
            item_name = driver.find_element(By.CSS_SELECTOR, 'h1.css-1n8byw.e14n6e5u1').text.strip()
            
            # 리뷰 페이지로 이동
            product_id = product_link.split("/")[-1]
            review_link = f"https://zigzag.kr/review/list/{product_id}"
            driver.get(review_link)
            time.sleep(2)    

            # 스크롤 다운 (3번)
            for _ in range(1):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                time.sleep(1) 
            
            # 리뷰 컨테이너 찾기 (리뷰 데이터 크롤링 시작)
            review_containers = driver.find_elements(By.CSS_SELECTOR, "div.css-vbvoj0.e13bai5o0")
            
            for review in review_containers:
                try:
                    reviewer = review.find_element(By.CSS_SELECTOR, "p.zds4_s96ru86.zds4_s96ru815").text.strip()
                    review_date = review.find_element(By.CSS_SELECTOR, "div.css-1xqlji6.eimmef70").text.strip()
                    
                    # 옵션 정보 가져오기
                    option_elements = review.find_elements(By.CSS_SELECTOR, "div.css-6e8rvi.eld8gav1")
                    options = [opt.text.strip() for opt in option_elements if opt.text.strip()]
                    
                    # 리뷰 
                    review_texts  = review.find_elements(By.CSS_SELECTOR, "span.zds4_s96ru86.zds4_s96ru813")
                    review_contents = [con.text.strip() for con in review_texts if con.text.strip()]
                    
                    # 정규표현식으로 값 추출
                    size = next((opt.split()[-1] for opt in options if "사이즈" in opt), "")
                    quality = next((opt.split()[-1] for opt in options if "퀄리티" in opt), "")
                    color = next((opt.split()[-1] for opt in options if "색감" in opt), "")
                    height = next((re.search(r"(\d+)cm", opt).group(1) + "cm" for opt in options if "cm" in opt), "")
                    weight = next((re.search(r"(\d+)kg", opt).group(1) + "kg" for opt in options if "kg" in opt), "")
                    top_size = next((opt.split()[-1] for opt in options if "상의" in opt), "")

                    # 데이터 CSV 파일에 저장
                    writer.writerow([brand_name, item_name, reviewer, review_date, size, quality, color, height, weight, top_size, review_contents])
                    
                    # 출력 확인
                    print(f"✔ 리뷰어: {reviewer} | 날짜: {review_date} | 사이즈: {size} | 퀄리티: {quality} | 색감: {color} | 키: {height} | 몸무게: {weight} | 상의: {top_size} | 리뷰: {review_contents}")
                
                except Exception as e:
                    print(f"⚠ 리뷰 크롤링 오류: {e}")
                    
            
        except Exception as e:
            print(f"⚠ '{product}' 검색 실패: {e}")


# 드라이버 종료
driver.quit()


In [21]:
import re
import pandas as pd

# 이모티콘 제거 함수 (대괄호 [] 는 남기고, 이모티콘만 제거)
def remove_emoji(text):
    # 이모티콘 범위에 해당하는 문자를 제거하는 정규식
    return re.sub(r'[^\w\s\[\],.-]', '', text)

# 데이터 불러오기
df = pd.read_csv(r'D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\top_info.csv', encoding='utf-8')
product_names = df['상품명'].tolist()

# product_names 리스트에서 각 상품명에서 이모티콘 제거
product_names = [remove_emoji(product) for product in product_names]

# 결과 확인
print(product_names)

['[MOMENT JOY국내제작라운드,스퀘어넥] 카엘리 쫀쫀 베이직 라운드 기본 이너 골지 민소매 단가라 나시', '[단독][made][5만장돌파셔츠BEST평점4.9극찬후기봄신상휴양지룩 여행룩] 프렌즈 루즈 체크셔츠 s158463', '키코 물결 펀칭 레이어드 긴팔 티셔츠', '릴렉스 심리스 브라 탑나시캡 탈부착 가능', '[무료배송][20쿠폰팩세일중][3만장돌파][인생하객룩저퀄카피주의]뉴컬러mm사이즈 단독제작 임프레션 튤립 스카프 블라우스 - 10color 긴팔버젼', '[핏극찬1.5만장돌파][MADE]뮤즈 봄 여름 오프숄더 드레이프 언발 셔링 반팔 티셔츠5color 군살커버팔뚝살커버하객룩여행룩휴양지룩데이트룩소개팅룩', '[봄버전] 이너 티셔츠 기본 무지 긴팔 모달 롱슬리브', '[MADE] 데일리 라운드넥 슬림 긴팔 티셔츠6차 재진행', '[햇볕차단] 쿨 루즈핏 시스루 거즈 포켓 셔츠 , 커버업 바캉스룩 얇은 살안타템 오버핏 린넨 수영복 가디건 여름 린넨 롤업 비치 휴가룩 휴양지룩 워터파크 긴팔 화이트 카키 블랙', '[캡내장] 쭈닝 쫀쫀 데일리 홀터 나시', '누적10만장 반팔긴팔 여유핏ver 삼육오 -3kg 유넥 부드러운 면 루즈핏 긴팔티 반팔티 5color [루즈핏면티폴라티여유핏반팔티면반팔여름반팔면긴팔기본긴팔]', '[레전드폴라귀환5만장돌파][2장특가컬러추가워머기모] 단독제작 허그 실루엣 기모 하프폴라 - 9color', '[4만장 돌파후기5천개이너티] 지니 반팔 티셔츠크롭반팔티유넥반팔티크롭티u넥반팔긴팔티셔츠이너티셔츠당일발송반팔흰티슬림반팔여름티반팔면티여름티셔츠검정반팔', '[20쿠폰팩11할인]캡내장 나시 크롭 이너 브라탑 캡나시 03', '[MADE한여름까지]선샤인 오버핏 체크셔츠', '[자체제작3만장돌파S-XL]봄 하모니 데일리 코튼 셔츠 데일리룩출근룩하객룩레이어드기본셔츠봄셔츠', '[MADE] 홀리스 브이넥 셔링 블라우스', '[극강의쫀쫀함부드러움][MADE] 베이직슬릿 레이온스판 긴팔티셔츠 44100 슬릿티-기본티-무지티-베이직티셔츠-데일리룩-스

In [ ]:
import time
import pandas as pd
import csv
import re
import os 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# 웹드라이버 실행 시 옵션 추가 (유니코드 문제 해결)
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드 (UI 없는 모드)
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--lang=ko-KR")  # 한국어 언어로 설정
chrome_options.add_argument("--no-sandbox")  # 보안 모드 비활성화
chrome_options.add_argument("--disable-dev-shm-usage")  # 메모리 관련 이슈 방지

# WebDriver 경로 지정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 웹드라이버 실행
url = 'https://zigzag.kr/search'
driver = webdriver.Chrome()

# 이모티콘 제거 함수 (대괄호 [] 는 남기고, 이모티콘만 제거)
def remove_emoji(text):
    # 이모티콘 범위에 해당하는 문자를 제거하는 정규식
    return re.sub(r'[^\w\s\[\],.-]', '', text)

# 데이터 불러오기
df = pd.read_csv(r'D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\top_info.csv', encoding='utf-8')
product_names = df['상품명'].tolist()

# product_names 리스트에서 각 상품명에서 이모티콘 제거
product_names = [remove_emoji(product) for product in product_names]
# 결과 확인
print(product_names)

# 파일이 없을 때만 새로 생성
file_path = r"D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\top_reviews.csv"

# 항상 새로 파일을 덮어쓰도록 수정
with open(file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["브랜드명", "상품명", "리뷰어", "날짜", "사이즈", "퀄리티", "색감", "키", "몸무게", "상의", "리뷰"])  
    print("📂 CSV 파일 덮어쓰기 완료!")

    for product in product_names:
        driver.get(url)
        time.sleep(2)  # 페이지가 완전히 로드될 시간을 충분히 줍니다.
    
        try:
            # 검색창 찾기
            search_box = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/main/div[1]/nav/div/input')
            search_box.clear()
            search_box.send_keys(product)
            search_box.send_keys(Keys.ENTER)
            time.sleep(2)  # 검색 후 페이지 로드 대기
            
            # 첫 번째 상품의 링크 가져오기
            link = driver.find_element(By.CSS_SELECTOR, '#__next > div > div.css-xrh5h3.edi339y0 > main > div.css-1ad8qtm.basic > div > div > div > div > div > div > div > a')
            product_link = link.get_attribute("href")
            driver.get(product_link)
            time.sleep(1)
            
            # 브랜드명, 상품명 가져오기
            brand_name = driver.find_element(By.CSS_SELECTOR, '.css-gwr30y').text.strip()
            item_name = driver.find_element(By.CSS_SELECTOR, 'h1.css-1n8byw.e14n6e5u1').text.strip()
            
            # 리뷰 페이지로 이동
            product_id = product_link.split("/")[-1]
            review_link = f"https://zigzag.kr/review/list/{product_id}"
            driver.get(review_link)
            time.sleep(2)    

            # 스크롤 다운 (1번)
            for _ in range(1):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                time.sleep(1) 
            
            # 리뷰 컨테이너 찾기 (리뷰 데이터 크롤링 시작)
            review_containers = driver.find_elements(By.CSS_SELECTOR, "div.css-vbvoj0.e13bai5o0")
            
            for review in review_containers:
                try:
                    reviewer = review.find_element(By.CSS_SELECTOR, "p.zds4_s96ru86.zds4_s96ru815").text.strip()
                    review_date = review.find_element(By.CSS_SELECTOR, "div.css-1xqlji6.eimmef70").text.strip()
                    
                    # 옵션 정보 가져오기
                    option_elements = review.find_elements(By.CSS_SELECTOR, "div.css-6e8rvi.eld8gav1")
                    options = [opt.text.strip() for opt in option_elements if opt.text.strip()]
                    
                    # 리뷰 내용 가져오기
                    review_texts  = review.find_elements(By.CSS_SELECTOR, "span.zds4_s96ru86.zds4_s96ru813")
                    review_contents = [con.text.strip() for con in review_texts if con.text.strip()]
                    
                    # 정규표현식으로 값 추출
                    size = next((opt.split()[-1] for opt in options if "사이즈" in opt), "")
                    quality = next((opt.split()[-1] for opt in options if "퀄리티" in opt), "")
                    color = next((opt.split()[-1] for opt in options if "색감" in opt), "")
                    height = next((re.search(r"(\d+)cm", opt).group(1) + "cm" for opt in options if "cm" in opt), "")
                    weight = next((re.search(r"(\d+)kg", opt).group(1) + "kg" for opt in options if "kg" in opt), "")
                    top_size = next((opt.split()[-1] for opt in options if "상의" in opt), "")

                    # 데이터 CSV 파일에 저장
                    writer.writerow([brand_name, item_name, reviewer, review_date, size, quality, color, height, weight, top_size, review_contents])
                    
                    # 출력 확인
                    print(f"✔ 리뷰어: {reviewer} | 날짜: {review_date} | 사이즈: {size} | 퀄리티: {quality} | 색감: {color} | 키: {height} | 몸무게: {weight} | 상의: {top_size} | 리뷰: {review_contents}")
                
                except Exception as e:
                    print(f"⚠ 리뷰 크롤링 오류: {e}")
                    continue
            
        except Exception as e:
            print(f"⚠ '{product}' 검색 실패: {e}")
            continue  # 예외 발생 시 다음 제품으로 넘어가도록 처리

# 드라이버 종료
driver.quit()


['MOMENT JOY국내제작라운드,스퀘어넥 카엘리 쫀쫀 베이직 라운드 기본 이너 골지 민소매 단가라 나시', '단독made5만장돌파셔츠BEST평점49극찬후기봄신상휴양지룩 여행룩 프렌즈 루즈 체크셔츠 s158463', '키코 물결 펀칭 레이어드 긴팔 티셔츠', '릴렉스 심리스 브라 탑나시캡 탈부착 가능', '무료배송20쿠폰팩세일중3만장돌파인생하객룩저퀄카피주의뉴컬러mm사이즈 단독제작 임프레션 튤립 스카프 블라우스  10color 긴팔버젼', '핏극찬15만장돌파MADE뮤즈 봄 여름 오프숄더 드레이프 언발 셔링 반팔 티셔츠5color 군살커버팔뚝살커버하객룩여행룩휴양지룩데이트룩소개팅룩', '봄버전 이너 티셔츠 기본 무지 긴팔 모달 롱슬리브', 'MADE 데일리 라운드넥 슬림 긴팔 티셔츠6차 재진행', '햇볕차단 쿨 루즈핏 시스루 거즈 포켓 셔츠 , 커버업 바캉스룩 얇은 살안타템 오버핏 린넨 수영복 가디건 여름 린넨 롤업 비치 휴가룩 휴양지룩 워터파크 긴팔 화이트 카키 블랙', '캡내장 쭈닝 쫀쫀 데일리 홀터 나시', '누적10만장 반팔긴팔 여유핏ver 삼육오 3kg 유넥 부드러운 면 루즈핏 긴팔티 반팔티 5color 루즈핏면티폴라티여유핏반팔티면반팔여름반팔면긴팔기본긴팔', '레전드폴라귀환5만장돌파2장특가컬러추가워머기모 단독제작 허그 실루엣 기모 하프폴라  9color', '4만장 돌파후기5천개이너티 지니 반팔 티셔츠크롭반팔티유넥반팔티크롭티u넥반팔긴팔티셔츠이너티셔츠당일발송반팔흰티슬림반팔여름티반팔면티여름티셔츠검정반팔', '20쿠폰팩11할인캡내장 나시 크롭 이너 브라탑 캡나시 03', 'MADE한여름까지선샤인 오버핏 체크셔츠', '자체제작3만장돌파SXL봄 하모니 데일리 코튼 셔츠 데일리룩출근룩하객룩레이어드기본셔츠봄셔츠', 'MADE 홀리스 브이넥 셔링 블라우스', '극강의쫀쫀함부드러움MADE 베이직슬릿 레이온스판 긴팔티셔츠 44100 슬릿티기본티무지티베이직티셔츠데일리룩스판티셔츠빅사이즈티셔츠출근룩체형커버', '반팔긴팔폴라 일년내내 3kg 부드러운 면 유넥 반팔티 긴팔티 col

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF67CFA4C25+3179557]
	(No symbol) [0x00007FF67CC088A0]
	(No symbol) [0x00007FF67CA98FFC]
	(No symbol) [0x00007FF67CADF8DF]
	(No symbol) [0x00007FF67CB17AF2]
	(No symbol) [0x00007FF67CB1244E]
	(No symbol) [0x00007FF67CB114F9]
	(No symbol) [0x00007FF67CA655E5]
	GetHandleVerifier [0x00007FF67D0067CD+3579853]
	GetHandleVerifier [0x00007FF67D01D1D2+3672530]
	GetHandleVerifier [0x00007FF67D012153+3627347]
	GetHandleVerifier [0x00007FF67CD7092A+868650]
	(No symbol) [0x00007FF67CC12FFF]
	(No symbol) [0x00007FF67CA641FF]
	GetHandleVerifier [0x00007FF67D08FA28+4141608]
	BaseThreadInitThunk [0x00007FFD75CA7374+20]
	RtlUserThreadStart [0x00007FFD760FCC91+33]


In [ ]:
import time
import pandas as pd
import csv
import re
import os 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# 웹드라이버 실행 시 옵션 추가 (유니코드 문제 해결)
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--lang=ko-KR")  # 한국어 언어로 설정
chrome_options.add_argument("--no-sandbox")  # 보안 모드 비활성화
chrome_options.add_argument("--disable-dev-shm-usage")  # 메모리 관련 이슈 방지


url = 'https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sort=200'
# WebDriver 경로 지정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# 스크롤하여 데이터 수집
scrolls = 1  # 원하는 만큼 스크롤 (조정 가능)
collected_links = set()  # 중복 방지를 위한 set()

for _ in range(scrolls):
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # 페이지가 로딩될 시간을 줌
    
    # 페이지 HTML 업데이트
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 상품 링크 수집 (새로운 상품이 로딩되었을 가능성이 있으므로 계속 추가)
    links = soup.select("#__next > div.zds-themes.light-theme > div.css-xrh5h3.edi339y0 > main > section.css-j2fe9p.eeob9mf0 > div a")
    
    for link in links:
        product_url = link.get('href')
        if product_url and product_url.startswith('/catalog/products/'):
            collected_links.add(f'https://zigzag.kr{product_url}')  # 중복 제거 위해 set에 추가

# 최종 수집된 링크 확인
print(f"총 {len(collected_links)}개의 상품 링크를 수집했습니다.")

# 수집된 정보를 저장할 리스트
data = []

# 상품 상세 정보 크롤링
for idx, full_url in enumerate(list(collected_links)):  # 모든 상품 링크를 크롤링
    driver.get(full_url)
    time.sleep(2)
    item_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        brand_name = item_page.select_one('.css-gwr30y').text.strip()
        item_name = item_page.select_one('h1.css-1n8byw.e14n6e5u1').text.strip()
        origin_price = item_page.select_one('.css-14j45be.e1yx2lle2').text.strip()
        discount_price = item_page.select_one('.css-no59fe.e1ovj4ty1').text.strip()
        review_count = item_page.select_one('span.css-1ovjo5n').text.strip()
        item_score_avg = item_page.select_one('.css-1hld56p.e71452m2').text.strip()
        brand_keyword = [span.text for span in item_page.select("#__next > div.zds-themes.light-theme > div.css-xrh5h3.edi339y0 > div > div.shop_row > div > div > div > div > div:nth-child(2) > div.css-2jjng0.eq4fxvi7 span")]
        
        print(brand_name, item_name, origin_price, discount_price, review_count, item_score_avg, brand_keyword, full_url)
        
        #리뷰 페이지 들어가기 
        product_id = full_url.split("/")[-1]
        # 리뷰 페이지 URL 생성
        review_link = f"https://zigzag.kr/review/list/{product_id}"
        driver.get(review_link)
        time.sleep(2)
        print(review_link)
        
        # 스크롤 다운 (3번)
        for _ in range(1):
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            time.sleep(2) 
        
        # 리뷰 컨테이너 찾기 (리뷰 데이터 크롤링 시작)
        review_containers = driver.find_elements(By.CSS_SELECTOR, "div.css-vbvoj0.e13bai5o0")
        
        for review in review_containers:
            try:
                #리뷰 더보기 버튼
                add_button = review.find_elements(By.CSS_SELECTOR, "span.zds4_s96ru86.zds4_s96ru813 p")
                
                for button in add_button:
                    try:
                        # 버튼이 클릭 가능한 상태인지 확인
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button))
                        
                        # ActionChains을 사용하여 클릭 시도
                        action = ActionChains(driver)
                        action.move_to_element(button).click().perform()
                        
                        time.sleep(1)
                    except Exception as e:
                        print(f"⚠ 버튼 클릭 오류: {e}")
                        
                reviewer = review.find_element(By.CSS_SELECTOR, "p.zds4_s96ru86.zds4_s96ru815").text.strip()
                review_date = review.find_element(By.CSS_SELECTOR, "div.css-1xqlji6.eimmef70").text.strip()
                
                # 옵션 정보 가져오기
                option_elements = review.find_elements(By.CSS_SELECTOR, "div.css-6e8rvi.eld8gav1")
                options = [opt.text.strip() for opt in option_elements if opt.text.strip()]
                
                # 리뷰 
                review_texts  = review.find_elements(By.CSS_SELECTOR, "span.zds4_s96ru86.zds4_s96ru813")
                review_contents = [con.text.strip() for con in review_texts if con.text.strip()]
                
                # 정규표현식으로 값 추출
                size = next((opt.split()[-1] for opt in options if "사이즈" in opt), "")
                quality = next((opt.split()[-1] for opt in options if "퀄리티" in opt), "")
                color = next((opt.split()[-1] for opt in options if "색감" in opt), "")
                height = next((re.search(r"(\d+)cm", opt).group(1) + "cm" for opt in options if "cm" in opt), "")
                weight = next((re.search(r"(\d+)kg", opt).group(1) + "kg" for opt in options if "kg" in opt), "")
                top_size = next((opt.split()[-1] for opt in options if "상의" in opt), "")
                
                # 출력 확인
                print(f"✔ 리뷰어: {reviewer} | 날짜: {review_date} | 사이즈: {size} | 퀄리티: {quality} | 색감: {color} | 키: {height} | 몸무게: {weight} | 상의: {top_size} | 리뷰: {review_contents}")
                
            
        
                # 수집한 데이터를 리스트에 추가
                data.append({
                    '브랜드': brand_name,
                    '상품명': item_name,
                    '원가': origin_price,
                    '할인 가격': discount_price,
                    '리뷰 개수': review_count,
                    '평점': item_score_avg,
                    '키워드': ', '.join(brand_keyword),
                    '링크': full_url,
                    '리뷰어': reviewer,
                    '작성날짜': review_date,
                    '사이즈': size,
                    '퀄리티': quality,
                    '색감': color,
                    '키': height,
                    '몸무게': weight,
                    '상의사이즈' : top_size,
                    '리뷰': review_contents
                })
                
                print(len(data))

            except Exception as e:
                        print(f"⚠ 리뷰 크롤링 오류: {e}")

    except AttributeError:
        print(f"[{idx+1}] 데이터가 부족한 상품입니다. URL: {full_url}")
    

# 크롤링된 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame을 CSV 파일로 저장
df.to_csv(r'D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\top_info_reviews.csv', index=False, encoding='utf-8')

# 브라우저 종료
driver.quit()

print(f"총 {len(df)}개의 상품 정보를 CSV로 저장했습니다.")